::::
:::{thebe-button}
:::
::::

# Fit tracks

Compare experimental bubble histories to bubble history correlations. Bubble detection and linking performed by Trackpy, an implementation of the Crocker-Grier algorithm {cite}`allanTrackpy2018,crockerMethodsDigitalVideo1996`.


In [ ]:
from boilercv_docs.nbs import init

paths = init()


from boilercore.fits import Fit, fit_from_params
from matplotlib.figure import Figure
from matplotlib.pyplot import subplots
from numpy import inf
from pandas import read_hdf, to_datetime
from seaborn import color_palette, lineplot, move_legend, scatterplot

from boilercv.correlations import SYMBOL_LABELS
from boilercv.dimensionless_params import jakob, prandtl
from boilercv_docs.nbs import HIDE, set_display_options
from boilercv_pipeline.experiments.e230920_subcool import MERGED_TRACKS, THERMAL_DATA

FIGURES: list[Figure] = []
LATENT_HEAT_OF_VAPORIZATION = 2.23e6  # J/kg
LIQUID_DENSITY = 960  # kg/m^3
LIQUID_DYNAMIC_VISCOSITY = 2.88e-4  # Pa-s
LIQUID_ISOBARIC_SPECIFIC_HEAT = 4213  # J/kg-K
LIQUID_THERMAL_CONDUCTIVITY = 0.676  # W/m-K
VAPOR_DENSITY = 0.804  # kg/m^3

MAX_REYNOLDS = 1500
MAX_NUSSELT = 50000

SCALE = 1.3
"""Plot scale."""
SIZE = 20 * SCALE
"""Size of the tracks."""
FONT_SCALE = SCALE
"""Font scale for plotting."""
WIDTH_SCALE = 2.00
"""Width scale for plotting."""
HEIGHT_SCALE = 1.30
"""Height scale for plotting."""


HIDE

In [ ]:
set_display_options(FONT_SCALE)
thermal_data = read_hdf(THERMAL_DATA)
tracks = (
    read_hdf(MERGED_TRACKS)
    .assign(**{"datetime": lambda df: to_datetime(df["datetime"])})
    .set_index("datetime")
    .assign(**{"Subcooling (K)": lambda df: thermal_data["subcool"]})
    .reset_index(drop=True)
    .assign(**{
        "Ja": lambda df: jakob(
            liquid_density=LIQUID_DENSITY,
            vapor_density=VAPOR_DENSITY,
            liquid_isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
            subcooling=df["Subcooling (K)"],
            latent_heat_of_vaporization=LATENT_HEAT_OF_VAPORIZATION,
        )
    })
).query("diameter_rate_of_change < 0")
constants = {
    "Pr": prandtl(
        dynamic_viscosity=LIQUID_DYNAMIC_VISCOSITY,
        isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
        thermal_conductivity=LIQUID_THERMAL_CONDUCTIVITY,
    ),
    "Ja": tracks["Ja"].mean(),
}

tracks

In [ ]:
def nusselt_model(Re_b, C_1, C_2, C_3, C_4):  # noqa: N803, D103
    return C_1 * Re_b**C_2 * constants["Pr"] ** C_3 * constants["Ja"] ** C_4


fits, errors = fit_from_params(
    model=nusselt_model,
    params=Fit(
        independent_params=["Re_b"],
        free_params=(params := ["C_1", "C_2", "C_3", "C_4"]),
        fixed_params=[],
        bounds=dict.fromkeys(params, (0, inf)),
        values=dict.fromkeys(params, 1),
    ),
    x=tracks["Re_b"].values,
    y=tracks["Nu_c"].values,
)

fits

In [ ]:
errors

In [ ]:
figure, ax = subplots()
FIGURES.append(figure)
ax.set_xlim(0, MAX_REYNOLDS)
ax.set_ylim(0, MAX_NUSSELT)
scatterplot(
    data=tracks.rename(columns=SYMBOL_LABELS),
    ax=ax,
    x="Bubble Reynolds number",
    y="Nusselt number",
    hue="Subcooling (K)",
    palette=color_palette("flare", as_cmap=True),
    s=SIZE,
)
data = (
    tracks.assign(**{"Model": nusselt_model(tracks["Re_b"], **fits)})
    .rename(columns=SYMBOL_LABELS)
    .set_index("Bubble Reynolds number")[["Model"]]
)
lineplot(ax=ax, data=data, legend=False)
move_legend(ax, "upper right")
HIDE

In [ ]:
figure, ax = subplots()
FIGURES.append(figure)
scatterplot(
    data=tracks.rename(columns=SYMBOL_LABELS),
    ax=ax,
    x="Bubble Reynolds number",
    y="Nusselt number",
    hue="Subcooling (K)",
    palette=color_palette("flare", as_cmap=True),
    s=SIZE,
)
ax.set(xscale="log", yscale="log")
ax.set_xlim(1e-14, 1e14)
ax.set_ylim(1e-17, 1e6)
move_legend(ax, "upper right")
HIDE